(assign3b_solution)=
# Assignment 3b -- Solution

1) In the cells below, get channel 32 and resample to the same area_def as channel 31
2) Get the brightness temperature for both of the resampled images and the brightness tempereature difference 
   for ch32 - ch31
3) Read in the wv_ir image you get by running the wv_resample notebook, and make a scatter plot of
   ch32 - ch31 brightness temperature (in K) on the y axis and the column water vapor in cm on the x axis.  Note that
   you will need to mask the brightness temperature pixels so that only pixels which also have column water vapor are
   plotted
4) comment on the correlation you see, if any

In [ ]:
import json
from pathlib import Path
import pprint
pp = pprint.PrettyPrinter(indent=4)
import warnings
warnings.filterwarnings('ignore')
import json


import matplotlib.pyplot as plt
import numpy as np
import a301_lib
from pyresample import kd_tree, SwathDefinition

import sat_lib
from sat_lib.modischan_read import readband_lw, read_plainvar
from sat_lib import modischan_dict
from rad_lib.radiation import radiance_invert

from sat_lib.mapping import area_def_from_dict

## Read in the channel 31 and 32 radiances and the 1 km MYD03 lons/lats

In [ ]:
### BEGIN SOLUTION
geom_filelist = list(a301_lib.sat_data.glob("pha/MYD03*2105*hdf"))
radiance_filelist = list(a301_lib.sat_data.glob("pha/MYD02*2105*hdf"))
geom_file_name = geom_filelist[0]
print(geom_file_name)
radiance_file_name = radiance_filelist[0]
print(radiance_file_name)
### END SOLUTION

## Read in the `wv_image` raster you stored in `wv_5km_resampled.npz`

- this is the file you created in week5/wv_resample.md

In [ ]:
### BEGIN SOLUTION
infile = a301_lib.data_share / "pha/wv_5km_resampled.npz"
the_dict = np.load(infile)
wv_image = the_dict['arr_0']
### END SOLUTION

## Recreate the area_def saved in your area_def json file 

- use `area_dict.json` for the 5km resample you created in
   week5/wv_resample.md and used in week5/longwave_resample.md

In [ ]:
### BEGIN SOLUTION
area_file = a301_lib.data_share / "pha/area_dict.json"
with open(area_file,'r') as infile:
    area_dict = json.load(infile)

pp.pprint(area_dict)   
area_def = area_def_from_dict(area_dict)
### END SOLUTION

## Get the swat_def from the 1 km lons/lats

In [ ]:
### BEGIN SOLUTION
lat_1km = read_plainvar(geom_file_name,'Latitude')
lon_1km = read_plainvar(geom_file_name,'Longitude')
print(f"{lat_1km.shape=},{lon_1km.shape=}")
swath_def = SwathDefinition(lon_1km, lat_1km)
### END SOLUTION

## Read in channel 31 and 32 resample using your `area_def` and `swath_def`

* As in in the wv_resample notebook, use a fill_value of -9999. and replace those
  missing values with np.nan

In [ ]:
### BEGIN SOLUTION
ch31 = readband_lw(radiance_file_name,31)
ch32 = readband_lw(radiance_file_name,32)
fill_value = -9999.
ch31_raster= kd_tree.resample_nearest(
    swath_def,
    ch31.ravel(),
    area_def,
    radius_of_influence=5000,
    nprocs=2,
    fill_value=fill_value,
)
ch31_raster[ch31_raster == -9999.0] = np.nan
ch32_raster= kd_tree.resample_nearest(
    swath_def,
    ch32.ravel(),
    area_def,
    radius_of_influence=5000,
    nprocs=2,
    fill_value=fill_value,
)
ch32_raster[ch32_raster == -9999.0] = np.nan
### END SOLUTION

## Convert the resampled ch31 and ch32 rasters to brightness temperature

- Remember that we have channel wavelenghts in `sat_lib.modischan_dict`
- I used [radiance_invert](https://phaustin.github.io/a301_web/full_listing.html#rad_lib.radiation.radiance_invert) here

In [ ]:
### BEGIN SOLUTION
low31,hi31 = modischan_dict['31']['wavelength_um']
center31=(low31 + hi31)/2.
low32,hi32 = modischan_dict['32']['wavelength_um']
center31=(low31 + hi31)/2.
center32=(low32 + hi32)/2.
Tbright31_raster = radiance_invert(center31*1.e-6,ch31_raster*1.e6)
Tbright32_raster = radiance_invert(center32*1.e-6,ch32_raster*1.e6)
### END SOLUTION

## Plot separate histograms of your Tbright31 and Tbright32 resampled rasters

In [ ]:
### BEGIN SOLUTION
bins = np.arange(220,350,10)
fig, ax = plt.subplots(1,1,figsize=(6,6))
ax.hist(Tbright31_raster.flat,alpha=0.4,color='blue',label='ch31',bins=bins)
ax.hist(Tbright32_raster.flat,alpha=0.4,color='red',label='ch32',bins=bins)
ax.legend();
### END SOLUTION

## make a scatterplot of (Tbright32 - Tbright31) vs. wv_image

* Plot only those points which have a valid water vapor retrieval using
a  mask like this:

    hit = wv_image > 0
    wv_image[hit], Tbright31_raster[hit], Tbright32_raster[hit]
    
* Put the MYD05 IR water vapor on x axis and the Tbright32 - Tbright31 brightness temperature
  difference on the y axis
  
* Include a title and x and y axis labels with units

In [ ]:
### BEGIN SOLUTION
hit = wv_image > 0
wv_good, Tb31good, Tb32good = wv_image[hit], Tbright31_raster[hit], Tbright32_raster[hit]
fig, ax = plt.subplots(1,1,figsize=(8,8))
ax.plot(wv_good, Tb32good - Tb31good,'r+')
ax.set(title = 'water vapor vs. Chan 32 - Chan 31 BTD for MODIS', xlabel='water vapor (cm/m^2)',
       ylabel = 'Channel 32 - Channel 31 BTD (K)');
### END SOLUTION
                

##  Comment on any correlation you see

Answer in the cell below (convert to markdown instead of python)
If you find a correlation, how does it compare with what you would expect for the dirty window?

Answer:  Channel 32 has more water vapor absorption, so increasing water vapor content increases the absorption in Channel 31.  Because the vapor is colder than the surface,'
emission to space is reduced, and the channel 31 brightness temperature is lower than
the surface temperature as seen in the clearer Channel 3..  This explains the negative values, and the fact that the negative magnitude becomes larger at large vapor concentrations.  There's a lot of scatter, however and the BTD rapidily becomes independent of water vapor for moister grid cells